# showq dynamic analysis

## Preprocessing

The output of Adaptive Computing Moab `showq` command has to be preprocessed by `scripts/showq2csv.py`. It will create a CSV file that can be imported by pandas.

## Prerequisites

Import required modules.

In [1]:
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
%matplotlib inline

Ensure that modules are reloaded automatically when modified.

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from lib.vsc.moab.nb_utils import extract_state

## Analysis

Load the current and the previous showq data.

In [7]:
curr_df = pd.read_csv('data/showq_1528968510.csv', parse_dates=['time_stamp', 'datetime'])

In [8]:
prev_df = pd.read_csv('data/showq_1528967910.csv', parse_dates=['time_stamp', 'datetime'])

Which jobs have started since the last showq epoch? These are the jobs that are currently active, and were previously either idle or blocked.

In [9]:
curr_running_df = extract_state(curr_df, 'Running')

In [10]:
prev_idle_def = extract_state(prev_df, 'Idle')

In [11]:
prev_blocked_df = extract_state(prev_df, 'Blocked')

In [21]:
curr_running_df.merge(prev_idle_def, on='job_id', how='inner', suffixes=('', '_prev'))[curr_running_df.columns]

,category,time_stamp,job_id,user_id,state,procs,remaining,start_time,walltime_limit
0,ActiveJob,2018-06-14 11:28:30,20842191,vsc31821,Running,20,35429,2018-06-14 11:19:22,35977.0
1,ActiveJob,2018-06-14 11:28:30,20842195,vsc31835,Running,80,172601,2018-06-14 11:25:34,172777.0


So two jobs that were idle are now running. Which jobs were blocked and are now running?

In [22]:
curr_running_df.merge(prev_blocked_df, on='job_id', how='inner', suffixes=('', '_prev'))[curr_running_df.columns]

,category,time_stamp,job_id,user_id,state,procs,remaining,start_time,walltime_limit


Which jobs were idle, and are now blocked?

In [17]:
curr_idle_df = extract_state(curr_df, 'Idle')

In [23]:
curr_idle_df.merge(prev_blocked_df, on='job_id', how='inner', suffixes=('', '_prev'))[curr_idle_df.columns]

,category,time_stamp,job_id,user_id,state,procs,walltime_limit,queue_time,time_in_queue


How many jobs are idle, and how many were idle in the previous epoch?

In [26]:
curr_idle_df.job_id.count(), prev_idle_def.job_id.count()

(117, 118)